===============================

### IMPORTS & GET DATABASE INFO

In [1]:
from jsons import read_json_to_dict
from mysql_driver import MySQL
import pandas as pd
from sqlalchemy import create_engine

json_readed = read_json_to_dict("sql_server_settings.json")
IP_DNS = json_readed["IP_DNS"]
USER = json_readed["USER"]
PASSWORD = json_readed["PASSWORD"]
BD_NAME = json_readed["BD_NAME"]
PORT = json_readed["PORT"]

In [2]:
# Connect to MySQL
mysql_db = MySQL(IP_DNS=IP_DNS, USER=USER, PASSWORD=PASSWORD, BD_NAME=BD_NAME, PORT=PORT)
mysql_db.connect()

Connected to MySQL server [datasciencetb_db]


============== 

### DROP TABLE

In [3]:
# Drop table if it already exist using execute() method.
#mysql_db.cursor.execute("DROP TABLE IF EXISTS people")

mysql_db.execute_interactive_sql(sql="DROP TABLE IF EXISTS people")

Executed 

DROP TABLE IF EXISTS people

 successfully


1

============== 

### CREATE TABLE

In [4]:
# Create table as per requirement
create_table_sql = """CREATE TABLE people(
    ID INT(11) NOT NULL AUTO_INCREMENT,
    MOMENTO TIMESTAMP NOT NULL,
    NOMBRE  VARCHAR(20) NOT NULL,
    APELLIDOS VARCHAR(100) NOT NULL,
    DIRECCION VARCHAR(50),
    EDAD INT,
    NOTA VARCHAR(40),
    PRIMARY KEY (ID))"""

mysql_db.execute_interactive_sql(sql=create_table_sql)

Executed 

CREATE TABLE people(
    ID INT(11) NOT NULL AUTO_INCREMENT,
    MOMENTO TIMESTAMP NOT NULL,
    NOMBRE  VARCHAR(20) NOT NULL,
    APELLIDOS VARCHAR(100) NOT NULL,
    DIRECCION VARCHAR(50),
    EDAD INT,
    NOTA VARCHAR(40),
    PRIMARY KEY (ID))

 successfully


1

============== 

### SELECT TABLE

In [5]:
# Select
select_sql = """SELECT * FROM people"""
select_result = mysql_db.execute_get_sql(sql=select_sql)

# tupla de tuplas
type(select_result)

Executing:
 SELECT * FROM people


tuple

In [6]:
select_result

()

============== 

### INSERT TABLE

In [7]:
# Insert

to_insert_1 = ["Pepito", "Wolfram_Eustaquio", "Calle Bellavista 9º-B", "67", "Enfermedad: Ceguera"]
to_insert_2 = ["Juanita", "Data Science", "Calle Recoletos", "15", "Está muy alegre siempre"]

sql_to_insert_1 = mysql_db.generate_insert_into_people_sql(to_insert=to_insert_1)
sql_to_insert_2 = mysql_db.generate_insert_into_people_sql(to_insert=to_insert_2)

In [8]:
sql_to_insert_1

"INSERT INTO people (MOMENTO, NOMBRE, APELLIDOS, DIRECCION, EDAD, NOTA) VALUES (NOW(), 'Pepito', 'Wolfram_Eustaquio', 'Calle Bellavista 9º-B', '67', 'Enfermedad: Ceguera')"

In [9]:
mysql_db.execute_interactive_sql(sql=sql_to_insert_1)
mysql_db.execute_interactive_sql(sql=sql_to_insert_2)

Executed 

INSERT INTO people (MOMENTO, NOMBRE, APELLIDOS, DIRECCION, EDAD, NOTA) VALUES (NOW(), 'Pepito', 'Wolfram_Eustaquio', 'Calle Bellavista 9º-B', '67', 'Enfermedad: Ceguera')

 successfully
Executed 

INSERT INTO people (MOMENTO, NOMBRE, APELLIDOS, DIRECCION, EDAD, NOTA) VALUES (NOW(), 'Juanita', 'Data Science', 'Calle Recoletos', '15', 'Está muy alegre siempre')

 successfully


1

=====================

### SELECT COLUMNS

In [10]:
select_result = mysql_db.execute_get_sql(sql=select_sql)
select_result

Executing:
 SELECT * FROM people


((1,
  datetime.datetime(2020, 10, 13, 20, 57, 51),
  'Pepito',
  'Wolfram_Eustaquio',
  'Calle Bellavista 9º-B',
  67,
  'Enfermedad: Ceguera'),
 (2,
  datetime.datetime(2020, 10, 13, 20, 57, 51),
  'Juanita',
  'Data Science',
  'Calle Recoletos',
  15,
  'Está muy alegre siempre'))

### Select with pandas

In [11]:
db_connection_str = mysql_db.SQL_ALCHEMY
db_connection = create_engine(db_connection_str)

df = pd.read_sql("select * from people", db_connection)
pd.set_option('display.expand_frame_repr', False)

df

,ID,MOMENTO,NOMBRE,APELLIDOS,DIRECCION,EDAD,NOTA
0,1,2020-10-13 20:57:51,Pepito,Wolfram_Eustaquio,Calle Bellavista 9º-B,67,Enfermedad: Ceguera
1,2,2020-10-13 20:57:51,Juanita,Data Science,Calle Recoletos,15,Está muy alegre siempre


### Insert from pandas

In [12]:
table_to_insert = "people"

df_to_insert = df.drop(columns=["ID"])
to_append = "append"
to_replace = "replace"

try:
    frame_sql  = df_to_insert.to_sql(name=table_to_insert, con=db_connection, if_exists=to_append, index=False)
    print("Success")
except Exception as error:
    print(error)

Success


In [13]:
# Select
select_sql = """SELECT * FROM people"""
select_result = mysql_db.execute_get_sql(sql=select_sql)

for row in select_result:
    ID = row[0]
    MOMENT = row[1]
    NAME = row[2]

    print("===========")
    print(ID, NAME)

Executing:
 SELECT * FROM people
1 Pepito
2 Juanita


=============================

### Drop row

In [15]:
sql_drop = """DELETE FROM people WHERE NOMBRE='Pepito';"""

mysql_db.execute_interactive_sql(sql=sql_drop)

Executed 

DELETE FROM people WHERE NOMBRE='Pepito';

 successfully


1

=============================

### Update row

In [16]:
sql_update = """UPDATE people set EDAD=102 WHERE NOMBRE='Juanita';"""

mysql_db.execute_interactive_sql(sql=sql_update)

Executed 

UPDATE people set EDAD=102 WHERE NOMBRE='Juanita';

 successfully


1

In [17]:
# Select
select_sql = """SELECT * FROM people"""
select_result = mysql_db.execute_get_sql(sql=select_sql)
select_result

Executing:
 SELECT * FROM people


((2,
  datetime.datetime(2020, 10, 13, 20, 58, 10),
  'Juanita',
  'Data Science',
  'Calle Recoletos',
  102,
  'Está muy alegre siempre'),
 (4,
  datetime.datetime(2020, 10, 13, 20, 58, 10),
  'Juanita',
  'Data Science',
  'Calle Recoletos',
  102,
  'Está muy alegre siempre'))

In [19]:
mysql_db.close()

Close connection with MySQL server [datasciencetb_db]
